In [2]:
#%pip install yt_dlp
import yt_dlp
import s3fs
import os

os.environ["AWS_ACCESS_KEY_ID"] = 'TFM0GSFSIN0GLN2OZMIK'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'qkhO8rOem3zuKQf21YuXMh14K5kJ7RB+RKf0DLrN'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJURk0wR1NGU0lOMEdMTjJPWk1JSyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzYyMzMxMjgxLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzYzODI1NjU5LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc2MzIyMDg1OSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDpmYzZkMTEzZC03MGE5LTBlNzItNzViNC03ZGRiZDRkYWU1MWQiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6ImNkZjM5NGE5LWQzZTMtNGZjZC05Y2E0LWUyNTU0ZTk5ODIwMyIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.cZHqmlEXSfirQKfAIlWGZw1kDYG5oV2uVwkZxTpjXgU9OR3L2g_iiNLvcFR4xPud14pSTJqchmEMRyZW5CyBKw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

In [ ]:
import yt_dlp
import re
import os
# import s3fs

# if using s3
# fs = s3fs.S3FileSystem()

ftv_url = "https://www.youtube.com/playlist?list=PLYXOi0ZKJEKKEWl_jgqTTXN9mzvcj4oP0" #FTV
esp_url = "https://www.youtube.com/playlist?list=PLMvsN4LH5cSFtlag_bbqDc76pyetG_4Ol" #ESP

def playlist_scraper (url) : 
    ydl_opts = {'quiet': True, 'extract_flat': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)

    playlist_dict = {entry['title']: entry for entry in info['entries']}

    # Start from index 340
    entries_to_download = list(playlist_dict.items())[886:]

    for title, entry in entries_to_download:
        # Skip long videos (>25 min)
        if entry.get('duration') and entry['duration'] > 1500:
            continue

        # Safe title for filenames
        safe_title = re.sub(r'[^a-zA-Z0-9_\- ]', '', title)[:100].strip()
        
        local_path = f"/tmp/{safe_title}.webm"
        
        # s3_path = 

        ydl_opts = {
            'format': 'bestaudio[ext=webm]',
            'outtmpl': local_path,
            'postprocessors': [],
            'quiet': True,
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([entry['url']])

        # Upload to S3
        # fs.put(local_path, s3_path)

        # Delete local temp file
        # os.remove(local_path)

playlist_scraper(ftv_url)
playlist_scraper(esp_url)